In [ ]:
Names: ['namestudent1', 'namestudent2'] 
Studentnumbers: ['studentnumberstudent1', 'studentnumberstudent2']

In [ ]:
# Download the data
# Dataset with 10K instances
! gdown "https://drive.google.com/uc?id=1cj-CzkY6QZUe42ky64GI5CSSg7-K40N5"
# Dataset with 100k instances
! gdown "https://drive.google.com/uc?id=1MbWGXLawE3VTxP1XgNpj8uEo1VHPq12B"

### Part 1: Import the data

# Applied Machine learning
## Practical Assignment 2

### Important Notes:
1. Submit through **Canvas** before 11:59pm on Tuesday, May 17, 2022
2. No late homework will be accepted
3. This is a group-of-two assignment
4. The submitted file should be in ipynb format
5. The assignment is worth it 10 points
6. For questions, please use the discussion part of Canvas (English only!)
7. The indication **optional** means that the question is optional; you won't lose any points if you do not do that part of the assignment, nor will you gain if you do it.

### Software:
We will be using Python programming language throughout this course. Further we will be using:
+ IPython Notebooks (as an environment)
+ Numpy
+ Pandas
+ Scikit-learn

### Background:

This practical assignment will be covering logistic regression, neural networks, support vector machines and evaluation of classifiers. 

For the assignment, please download a dataset on Load Defaults. You are provided with two datasets:
1. [Dataset](https://drive.google.com/open?id=1cj-CzkY6QZUe42ky64GI5CSSg7-K40N5) with 10,000 instances 
2. [Dataset](https://drive.google.com/open?id=1MbWGXLawE3VTxP1XgNpj8uEo1VHPq12B) with 100,000 instances

In principle you should work on the second, larger dataset, but if you face scaling computational issues then better work with the first, smaller dataset.

This data corresponds to a set of financial transactions associated with individuals. The data has been standardized, de-trended, and anonymized. You are provided with thousands of observations and nearly 800 features. Each observation (instance) is independent from the previous. 

For each observation, it was recorded whether a default was triggered (i.e. whether an individual could not pay back the loan). In case of a default, the loss for the bank was measured. This quantity lies between 0 and 100. If the loan did not default, the loss for the bank was 0. You are asked to predict whether a loan will default or not for each individual in the test set.

Missing feature values have been kept as is, so that the competing teams can really use the maximum data available, implementing a strategy to fill the gaps if desired. Consider all variables continuous, even though some variables may be categorical (e.g. f776 and f777).

The goal of the machine learning algorithm will be to predict whether a loan will default, given a set of features. For privacy reasons the feature names are not provided.

**Important Note**: This second assignment is not as instructive as the first assignment. The first assignment guided you step-by-step through all the preprocessing, training-validation-testing setup, etc. This assignment does not do so, but it leaves it up to you to decide how to use the data and design your experiments.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('loan_default_10K.csv', sep=",", header=0, dtype=np.float64)

# Drop the observations that contain missing values
dfn = df.dropna(0, how='any')

# Consider only a handful of features to start with; you can extend to the full set later on.
X = dfn.loc[:,'f400':'f500'].values

# Generate the labels; if 'loss' is zero the this indicates the negative class, class 0, i.e. no default;
# if 'loss' is possitive this indicates the positive class, class 1, i.e. there is a loan default;
y = [ bool(y) for y in dfn.loc[:,'loss'].values ]

# Split the data into train, validation, and test.
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20)

### Part 2: Evaluation measures (2pts)
In what follows you should implement a number of evaluation measures. You need to implement these from scratch, meaning that it is not allowed to call any scikit-learn function, or any other API function that implements the method for you.

* Implement a function that produces the contigency matrix, i.e. True Positives, False Positives, True Negatives, False Negative

In [ ]:
def contigency_matrix(true_y, predicted_y):
    # YOUR CODE HERE, Create TP, FP, TN, FN
    
    
    # Make sure your output fits the following format:
    # matrix = np.array(([TP, FP], [TN, FN]))
    return matrix

* Implement a function that computes accuracy (without using any built-in accuracy function)

In [ ]:
def accuracy(true_y, predicted_y):
    # YOUR CODE HERE
    

* Implement a function that computes precision for one class (without using any built-in precision function)

In [ ]:
def precision(true_y, predicted_y):
    # YOUR CODE HERE
    

* Implement a function that computes recall for one class (without using any built-in recall function)

In [ ]:
def recall(true_y, predicted_y):
    # YOUR CODE HERE
   

* Implement a function that computes f1 for one class(without using any built-in f1 function)

In [ ]:
def f1(true_y, predicted_y):
    # YOUR CODE HERE
    

### Part 3: Algorithms
Compare the performance of Logistic Regression, SVMs and Neural Networks

##### Logistic Regression (Lecture 3) (2pts)

+ Train and test a logistic regression model
    + Construct a table with each row being a different value of the regularization parameter and each column the aforementioned measures
    + Explain your findings and select the optimal model
    + Report the performance of the optimal model

In [ ]:
# YOUR CODE HERE


* Explain what you observe regarding the positive class; i.e. the performance of the algorithm in predicting defaults. Explain why is this happening.

<span style="color:blue">**Replace the text in this cell with your explanation**</span>

There are a number of ways to fix the problem you have observed above. Here we will consider two of them: downsampling and upsampling. In an ideal situation you will like your dataset to be balanced, i.e. to have the same number of instances for the positive and the negative class.

**Downsampling**: Let's assume that the positive class has *n1* instances, while the negative class *n2* instances, where *n2* is much bigger than *n1*. One solution is to create a new training set for which from the *n2* instances of the negative class you sample *n1* of them only to include in your training set; hence now you have *n1* + *n1* training instances.

**Upsampling**: Let's assume that the positive class has *n1* instances, while the negative class *n2* instances, where *n2* is much bigger than *n1*. Another solution is to create a new training set for which you create  *n2* instances of the positive class. To do so you sample *n2* instances from the *n1* instance, with replacement. With replacement means that you allow the same instance to be sampled multiple times; hence now you have *n2* + *n2* training instances.

#### Downsampling (OPTIONAL – If you wish to skip downsampling continue to Neural Networks further below)


* Implement a function for downsampling (**optional**)

In [ ]:
def downsample(y_train):
    # y_train is the 1d matrix of the labels in your training data, e.g.
    #       0     1     2     3     4   5     6     7     8   ... 
    # y = [True False False False True True False False False ... False]
    #
    # the function returns the position of the training data to be considered for the final training set.
    # e.g. if you decide from the True instances to select 0, 4 and 5, while from the False instances 1, 3, and 8
    # the outcome of the function will be [0, 1, 3, 4, 5, 8] (= sampled_indexes)
    
    # your code goes here
    
    return sampled_indexes
    
def new_training_set(X_train, y_train, sampled_indexes):
    # your code goes here

* Test the performance of logistic regression using the new training set, and report your conclusions (**optional**)

In [ ]:
# YOUR CODE HERE

# The last few questions below are not optional!
If you did not finish the optional downsampling, just go through with the data created before

##### SVMs (Lecture 4) (2pts)

+ Train and test a Support Vector Machine model
    + Construct a table with each row being a different configuration of the SVM algorithm (play with the regularization parameter, and the kernel function – use linear, poly, and rbf) and each column the evaluation measures
    + Explain your findings and select the optimal model
    + Report the performance of the optimal model

In [ ]:
# YOUR CODE HERE

<span style="color:blue">**Replace the text in this cell with your report**</span>

##### Neural Network (Lecture 5) (2pts)

+ Train and test a Neural Network model
    + Construct a table with each row being a different configuration of the network (play with the number of hidden layers, the number of neurons in each layer, and the activation function) and each column the evaluation measures
    + Report the performance of at least three different configurations
    + Explain your findings and select the optimal model
    + Justify your choice of different paramteres and architectures
    + Report the performance of the optimal model

In [ ]:
# YOUR CODE HERE

<span style="color:blue">**Replace the text in this cell with your report**</span>

#### Compare Algorithms (2pts)
* Plot the Precision-Recall curves for the best model for each one of the above algorithms, Logistic Regression, Neural Nets, and SVM.
    * Use the precision_recall_curve from scikit-learn
* Explain your findings

In [ ]:
# YOUR CODE HERE

<span style="color:blue">**Replace the text in this cell with your report**</span>